# NASA Spacecraft Telemetry Data

In [22]:
import numpy as np
import pandas as pd
import json
import os
from typing import Final, List, Callable
from config import data_raw_folder, data_processed_folder
from timeeval import Datasets, DatasetRecord
from timeeval.datasets import DatasetAnalyzer

In [23]:
dataset_collection_prefix = "NASA"
source_folder = os.path.join(data_raw_folder, "NASA Spacecraft Telemetry Data")
target_folder = data_processed_folder

from pathlib import Path
print(f"Looking for source datasets in {Path(source_folder).absolute()} and\nsaving processed datasets in {Path(target_folder).absolute()}")

Looking for source datasets in /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data and
saving processed datasets in /home/projects/akita/data/benchmark-data/data-processed


In [30]:
def create_target_subfolder(input_type: str, dataset_collection_name: str) -> str:
    dataset_subfolder = os.path.join(input_type, dataset_collection_name)
    target_subfolder = os.path.join(target_folder, dataset_subfolder)
    try:
        os.makedirs(target_subfolder)
        print(f"Created directories {target_subfolder}")
    except FileExistsError:
        print(f"Directories {target_subfolder} already exist")
        pass

    return dataset_subfolder, target_subfolder

def transform_and_label(source: str, target: str, anomaly_windows: List[str], force_all_normal: bool = False) -> None:
    df = pd.DataFrame(np.load(source)[:,0], columns=["value"])
    df = df[["value"]]
    df.insert(0, "timestamp", df.index.values)
    df["is_anomaly"] = 0

    if not force_all_normal:
        for t1, t2 in anomaly_windows:
            tmp = df[df.index >= t1]
            tmp = tmp[tmp.index <= t2]
            df["is_anomaly"].values[tmp.index] = 1

    df.to_csv(target, index=False)
    return df

In [31]:
# shared by all datasets
dataset_type = "real"
input_type = "univariate"
datetime_index = False
train_type = "semi-supervised"
train_is_normal = True

dm = Datasets(target_folder)

In [32]:
# dataset transformation
transform_file: Callable[[str, str, List[str], bool], None] = transform_and_label

meta = pd.read_csv(os.path.join(source_folder, "labeled_anomalies.csv"))

for _, dataset in meta.iterrows():
    
    dataset_name = dataset["chan_id"]
    collection_name = dataset_collection_prefix + "-" + dataset["spacecraft"]
    dataset_length = dataset["num_values"]
    dataset_subfolder, target_subfolder = create_target_subfolder(input_type, collection_name)
    
    windows = json.loads(dataset["anomaly_sequences"])
    
    meta = None
    paths = {}
    for t_type in ["train", "test"]:
        source_file = os.path.join(source_folder, t_type, dataset_name + ".npy")
        filename = f"{dataset_name}.{t_type}.csv"
        path = os.path.join(dataset_subfolder, filename)
        target_filepath = os.path.join(target_subfolder, filename)
        meta_target_filepath = os.path.join(target_subfolder, f"{dataset_name}.{Datasets.METADATA_FILENAME_PREFIX}")
        paths[t_type] = path
        
        # transform file
        df = transform_file(source_file, target_filepath, windows, force_all_normal=(t_type == "train"))
        da = DatasetAnalyzer((collection_name, dataset_name), is_train=(t_type == "train"), df=df)
        da.save_to_json(meta_target_filepath, overwrite=(t_type != "train"))
        if t_type != "train":
            meta = da.metadata
        print(f"Processed source dataset {source_file} -> {target_filepath}")

    # save metadata
    dm.add_dataset(DatasetRecord(
        collection_name=collection_name,
        dataset_name=dataset_name,
        train_path=paths["train"],
        test_path=paths["test"],
        dataset_type=dataset_type,
        datetime_index=datetime_index,
        split_at=None,
        train_type=train_type,
        train_is_normal=train_is_normal,
        input_type=input_type,
        length=meta.length,
        dimensions=meta.dimensions,
        contamination=meta.contamination,
        num_anomalies=meta.num_anomalies,
        min_anomaly_length=meta.anomaly_length.min,
        median_anomaly_length=meta.anomaly_length.median,
        max_anomaly_length=meta.anomaly_length.max,
        mean=meta.mean,
        stddev=meta.stddev,
        trend=meta.trend,
        stationarity=meta.get_stationarity_name(),
    ))

# save metadata of benchmark
dm.save()

Directories /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP already exist
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/train/P-1.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP/P-1.train.csv


[('NASA-SMAP', 'P-1') (test)] /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP/P-1.metadata.json already exists, but 'overwrite' was specified! Ignoring existing contents.


Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/test/P-1.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP/P-1.test.csv
Directories /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP already exist
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/train/S-1.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP/S-1.train.csv


[('NASA-SMAP', 'S-1') (test)] /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP/S-1.metadata.json already exists, but 'overwrite' was specified! Ignoring existing contents.


Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/test/S-1.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP/S-1.test.csv
Directories /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP already exist
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/train/E-1.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-SMAP/E-1.train.csv


...

Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/train/D-15.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/D-15.train.csv
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/test/D-15.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/D-15.test.csv
Directories /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL already exist
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/train/D-16.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/D-16.train.csv


[('NASA-MSL', 'D-16') (test)] /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/D-16.metadata.json already exists, but 'overwrite' was specified! Ignoring existing contents.
[('NASA-MSL', 'M-7') (test)] /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/M-7.metadata.json already exists, but 'overwrite' was specified! Ignoring existing contents.


Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/test/D-16.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/D-16.test.csv
Directories /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL already exist
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/train/M-7.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/M-7.train.csv
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/test/M-7.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/M-7.test.csv
Directories /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL already exist
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/train/F-8.npy -> /home/projects/akita/data/benchmark-

[('NASA-MSL', 'F-8') (test)] /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/F-8.metadata.json already exists, but 'overwrite' was specified! Ignoring existing contents.


Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/NASA Spacecraft Telemetry Data/test/F-8.npy -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NASA-MSL/F-8.test.csv


In [33]:
dm.refresh()
dm.df().loc[(slice("NASA-MSL", "NASA-SMAP"), slice(None))]

train_path  \
collection_name dataset_name                                       
NASA-MSL        C-1            univariate/NASA-MSL/C-1.train.csv   
                C-2            univariate/NASA-MSL/C-2.train.csv   
                D-14          univariate/NASA-MSL/D-14.train.csv   
                D-15          univariate/NASA-MSL/D-15.train.csv   
                D-16          univariate/NASA-MSL/D-16.train.csv   
...                                                          ...   
NASA-SMAP       R-1           univariate/NASA-SMAP/R-1.train.csv   
                S-1           univariate/NASA-SMAP/S-1.train.csv   
                T-1           univariate/NASA-SMAP/T-1.train.csv   
                T-2           univariate/NASA-SMAP/T-2.train.csv   
                T-3           univariate/NASA-SMAP/T-3.train.csv   

                                                      test_path dataset_type  \
collection_name dataset_name                                                   
NASA-MSL        C-1            univariate/NASA-MSL/C-1.test.csv         real   
                C-2            univariate/NASA-MSL/C-2.test.csv         real   
                D-14          univariate/NASA-MSL/D-14.test.csv         real   
                D-15          univariate/NASA-MSL/D-15.test.csv         real   
                D-16          univariate/NASA-MSL/D-16.test.csv         real   
...                                                         ...          ...   
NASA-SMAP       R-1           univariate/NASA-SMAP/R-1.test.csv         real   
                S-1           univariate/NASA-SMAP/S-1.test.csv         real   
                T-1           univariate/NASA-SMAP/T-1.test.csv         real   
                T-2           univariate/NASA-SMAP/T-2.test.csv         real   
                T-3           univariate/NASA-SMAP/T-3.test.csv         real   

                              datetime_index  split_at       train_type  \
collection_name dataset_name                                              
NASA-MSL        C-1                    False       NaN  semi-supervised   
                C-2                    False       NaN  semi-supervised   
                D-14                   False       NaN  semi-supervised   
                D-15                   False       NaN  semi-supervised   
                D-16                   False       NaN  semi-supervised   
...                                      ...       ...              ...   
NASA-SMAP       R-1                    False       NaN  semi-supervised   
                S-1                    False       NaN  semi-supervised   
                T-1                    False       NaN  semi-supervised   
                T-2                    False       NaN  semi-supervised   
                T-3                    False       NaN  semi-supervised   

                              train_is_normal  input_type  length  dimensions  \
collection_name dataset_name                                                    
NASA-MSL        C-1                      True  univariate    2264           1   
                C-2                      True  univariate    2051           1   
                D-14                     True  univariate    2625           1   
                D-15                     True  univariate    2158           1   
                D-16                     True  univariate    2191           1   
...                                       ...         ...     ...         ...   
NASA-SMAP       R-1                      True  univariate    7244           1   
                S-1                      True  univariate    7331           1   
                T-1                      True  univariate    8612           1   
                T-2                      True  univariate    8625           1   
                T-3                      True  univariate    8579           1   

                              contamination  num_anomalies  \
collection_name dataset_name        

## Experimentation

In [ ]:
meta = pd.read_csv(os.path.join(source_folder, "labeled_anomalies.csv"))
json.loads(meta["anomaly_sequences"][0])
dataset = next(meta.iterrows())[1]
print(dataset)

In [ ]:
columns = ["value"] + ["C" + str(i) for i in range(24)]
filename = os.path.join(source_folder, "test", dataset["chan_id"] + ".npy")
print(f"loading {filename}")
dd = np.load(filename)
df = pd.DataFrame(dd, columns=columns)
df.index.name = "timestamp"
df = df[["value"]]
df["is_anomaly"] = 0

windows = json.loads(dataset["anomaly_sequences"])

for t1, t2 in windows:
    tmp = df[df.index >= t1]
    tmp = tmp[tmp.index <= t2]
    df["is_anomaly"].values[tmp.index] = 1

print(windows)
df

In [ ]:
trainfile = os.path.join(source_folder, "train", dataset["chan_id"] + ".npy")
testfile = os.path.join(source_folder, "test", dataset["chan_id"] + ".npy")
windows = json.loads(dataset["anomaly_sequences"])

transform_and_label(trainfile, "P-1.train.csv", windows, force_all_normal=True)
transform_and_label(testfile, "P-1.test.csv", windows, force_all_normal=False)

df = pd.read_csv("P-1.train.csv", index_col="timestamp")
anomalies_in_train = len(df[df["is_anomaly"] == 1])
df = pd.read_csv("P-1.test.csv", index_col="timestamp")
anomalies_in_test = len(df[df["is_anomaly"] == 1])
(anomalies_in_train, anomalies_in_test)

In [35]:
from pathlib import Path
df = pd.read_csv(Path(target_folder) / "univariate" / "NASA-MSL" / "C-1.train.csv")
df

,timestamp,value,is_anomaly
0,0,2.146646,0
1,1,2.146646,0
2,2,2.146646,0
3,3,2.151326,0
4,4,2.163807,0
...,...,...,...
2153,2153,-0.996880,0
2154,2154,-0.998440,0
2155,2155,-0.982839,0
2156,2156,-0.976599,0


In [37]:
from pathlib import Path
np.load(Path(source_folder) / "train" / "P-1.npy")[:,0]

array([-0.52315476, -0.68885677, -0.55499272, ..., -0.72575974,
       -0.68958026, -0.70984087])